In [1]:
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [2]:
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class WholeAutoAnnotationsClusters3(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> pinky.PymeshfixDecimatedExcitatoryStitchedMesh
    ---
    vertices   : longblob     # label data for the vertices
    triangles  : longblob     # label data for the faces
    """
    
    def make(self, key):
        start_time = time.time()
        
        print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")
        
        segment_id = key["segment_id"]

        #get the vertices and faces from datajoint
        # get the newly stitched mesh
        # get the original mesh

        
        verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")


        #run the whole algorithm on the neuron to test
        verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                             triangles=faces,pymeshfix_Flag=False,
                             import_CGAL_Flag=False,
                             return_Only_Labels=True,
                             clusters=3,
                             smoothness=0.2)
        
        #insert the key
        self.insert1(dict(key,
                  vertices=verts_labels,
                  triangles=faces_labels),skip_duplicates=True)


        
        print(f"Finished Auto Labeling: {time.time() - start_time}")
        

In [4]:
start = time.time()
WholeAutoAnnotationsClusters3.populate(reserve_jobs=True)
print(time.time() - start)



*****Starting Auto Labeling for 648518346349498116******
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.1308445930480957
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2
1
Finished CGAL segmentation algorithm: 1552.7456469535828
2) Finished: Generating CGAL segmentation for neuron: 1577.9754693508148
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 0
3) Finished: Generating Graph Structure and Identifying Soma: 0.29873061180114746
5) Staring: Finding Apical Index
Soma Index = 0
Soma Connections = [13, 31, 6, 8, 30, 52, 25, 36, 33, 18, 68, 92, 69, 159]
soma_80_percent = 243372.703125
Debugging the axon filter
[(13, 266096.53125), (31, 249715.921875), (6, 243247.6875), (8, 239990.765625), (30, 259521.78125), (52, 253223.96875), (25, 255385.1875), (36, 260892.18

In [5]:
#(schema.jobs & "table_name='__whole_auto_annotations_clusters3'").delete()

In [6]:
#schema.jobs